In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split

In [ ]:
train=pd.read_csv('/content/train.csv')

test=pd.read_csv('/content/test.csv')

df=pd.concat((train.iloc[:,1:-1],test.iloc[:,1:]))
df=pd.get_dummies(df)
df.columns

Index(['date_2015-01-01', 'date_2015-01-02', 'date_2015-01-03',
       'date_2015-01-04', 'date_2015-01-05', 'date_2015-01-06',
       'date_2015-01-07', 'date_2015-01-08', 'date_2015-01-09',
       'date_2015-01-10',
       ...
       'date_2019-12-30', 'date_2019-12-31', 'country_Finland',
       'country_Norway', 'country_Sweden', 'store_KaggleMart',
       'store_KaggleRama', 'product_Kaggle Hat', 'product_Kaggle Mug',
       'product_Kaggle Sticker'],
      dtype='object', length=1834)

In [ ]:
train_1=train.shape[0]
x=df[:train_1]
y=train['num_sold']
y=np.log1p(y)
#x=x[['country_Norway', 'country_Sweden','store_KaggleRama', 'product_Kaggle Hat','product_Kaggle Mug']]

test_set=df[train_1:]
#test_set=test_set[['country_Norway', 'country_Sweden','store_KaggleRama', 'product_Kaggle Hat','product_Kaggle Mug']]
x.shape

(26298, 1834)

In [ ]:
y.shape

(26298,)

In [ ]:
test_set.shape

(6570, 1834)

In [ ]:
plt.figure(figsize=(10,8))
sns.scatterplot(data=train,x='date',y='num_sold')
plt.show()

In [ ]:
train.head()

In [ ]:
test.head()

In [ ]:
train.columns

In [ ]:
train.info()

In [ ]:
import tensorflow as tf
from tensorflow import feature_column
from tensorflow.keras import layers
from tensorflow.keras.layers import DenseFeatures,Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import StandardScaler

In [ ]:
x=train.drop(['row_id','num_sold'],axis=1)
y=train['num_sold']

In [ ]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=32)
standard_scaler = StandardScaler()
x_train_scaled = pd.DataFrame(
      standard_scaler.fit_transform(x_train),
      columns=x_train.columns
  )
x_test_scaled = pd.DataFrame(
      standard_scaler.transform(x_test),
      columns = x_test.columns
  )
test_set_scaled = pd.DataFrame(
      standard_scaler.transform(test_set),
      columns = test_set.columns
  )



In [ ]:
hidden_units1 = 200
hidden_units2 = 300
hidden_units3 = 150
learning_rate = 0.01
# Creating model using the Sequential in tensorflow
def build_model_using_sequential():
  model = tf.keras.models.Sequential([                             
    tf.keras.layers.Dense(hidden_units1,  activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(hidden_units2,  activation='relu'),
    Dropout(0.2),
    tf.keras.layers.Dense(hidden_units3,  activation='relu'),
    tf.keras.layers.Dense(1, activation='linear')
  ])
  return model
# build the model
model = build_model_using_sequential()

In [ ]:
from tensorflow.keras.losses import MeanSquaredError
mse=MeanSquaredError()
model.compile(loss=mse,optimizer=Adam(lr=0.01))
# train the model
history = model.fit(
    x_train_scaled.values, 
    y_train.values, 
    epochs=30,
    batch_size=64,
    validation_split=0.2
)

In [ ]:
y_hat = model.predict(x_test_scaled)

In [ ]:
from sklearn.metrics import r2_score
print(r2_score(y_test,y_hat))

In [ ]:
y_pred=model.predict(test_set_scaled)

In [ ]:
test['num_sold']=pd.Series(y_pred.reshape(1,-1)[0])
submission=pd.concat([test['row_id'],test['num_sold']],axis=1)
submission.to_csv('submission6.csv',index=False)

In [ ]:
#x_df=train.drop(['row_id'],axis=1)
#x_df=pd.get_dummies(x_df)

In [ ]:
#x_df.columns

In [ ]:
x=x_df
y=x_df['num_sold']
y=np.lop1p(y)

In [ ]:
plt.figure(figsize=(10,8))
sns.heatmap(x_df.corr(),annot=True)
plt.show()

In [ ]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=32)

In [ ]:
plt.figure(figsize=(10,8))
sns.scatterplot(data=x_df,x='product_Kaggle Hat',y='num_sold',palette='crest')
plt.show()

In [ ]:
plt.figure(figsize=(10,8))
sns.scatterplot(data=x_df,x='store_KaggleRama',y='num_sold',palette='crest')
plt.show()

In [ ]:
plt.figure(figsize=(10,8))
sns.scatterplot(data=x_df,x='country_Norway',y='num_sold',palette='crest')
plt.show()

In [ ]:
x_df['num_sold'].head()

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
lr=LinearRegression()
lr.fit(x_train,y_train)
y_hat1=lr.predict(x_test)
print(r2_score(y_test,y_hat1))

In [ ]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import r2_score
dt=DecisionTreeRegressor(max_depth=4,random_state=32)
dt.fit(x_train,y_train)
y_hat2=dt.predict(x_test)
print(r2_score(y_test,y_hat2))

In [ ]:
from sklearn.ensemble import RandomForestRegressor
rf=RandomForestRegressor(n_estimators=500,max_leaf_nodes=16,n_jobs=-1)
rf.fit(x_train,y_train)
y_hat3=rf.predict(x_test)
print(r2_score(y_test,y_hat3))

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
gb=GradientBoostingRegressor(max_depth=4,n_estimators=120,random_state=32)
gb.fit(x_train,y_train)
y_hat4=gb.predict(x_test)
print(r2_score(y_test,y_hat4))

0.9132767148848641


In [ ]:
from sklearn.ensemble import AdaBoostRegressor
ada=AdaBoostRegressor(DecisionTreeRegressor(),n_estimators=200,learning_rate=0.01)
ada.fit(x_train,y_train)
y_hat5=ada.predict(x_test)
print(r2_score(y_test,y_hat5))

In [ ]:
import xgboost
from xgboost import XGBRegressor
xgb=XGBRegressor()
xgb.fit(x_train,y_train)
y_hat6=xgb.predict(x_test)
print(r2_score(y_test,y_hat6))

[08:17:16] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
0.9091926736331554


In [ ]:
test=pd.get_dummies(test)
test.columns

      

In [ ]:
y_pred=gb.predict(test_set)
test['num_sold']=pd.Series(y_pred.reshape(1,-1)[0])
submission=pd.concat([test['row_id'],test['num_sold']],axis=1)
submission.to_csv('submission7.csv',index=False)